### Merging and Cleaning the Dataset for Analysis

In [2]:
import censusdata 
import pandas as pd 
import numpy as np
import math

### 1. Downloading Census Data 

In [3]:
#Use censusdata module to read in acs data 
ml_features = censusdata.download("acs5", 2018, censusdata.censusgeo(
    [("county", "*")]), ["B01003_001E", "B02001_001E", "B02001_002E", "B19301_001E","C17002_008E",
                                         "C17002_004E", "C17002_005E", "C17002_006E", "C17002_007E", "C17002_008E", 
                                         "B15003_022E", "B16005_007E", "B16005_008E", "B16005_012E", "B16005_013E",
                                         "B16005_027E", "B16005_018E", "B16005_022E", "B16005_023E", "B16005_029E", "B16005_030E",
                                         "B16005_034E", "B16005_035E", "B16005_039E", "B16005_040E", "B16005_044E", "B16005_045E",
                                         "GEO_ID", "B23001_001E"])


var_dict = {"B01003_001E": "Total Population", "B02001_002E": "White_Alone", "C17002_008E": "ratio_income_poverty (above 200% FPL)", 
           "B15003_022E": "Total Bachelors Degree", "B19301_001E": "Per Capita Income"} 

In [4]:
#Rename variables for clarity 
ml_feat = ml_features.rename(columns=var_dict)

In [5]:
#engineer features from acs data 
ml_feat['Minority Population'] = ml_feat['Total Population'] - ml_feat["White_Alone"]
ml_feat['Est Speak English Less than Well'] = ml_feat["B16005_007E"] + ml_feat["B16005_008E"] + ml_feat["B16005_012E"] + ml_feat["B16005_013E"] + \
                                              ml_feat["B16005_027E"] + ml_feat["B16005_018E"] + ml_feat["B16005_022E"] + ml_feat["B16005_023E"] + \
                                              ml_feat["B16005_029E"] + ml_feat["B16005_030E"] + ml_feat["B16005_034E"] + \
                                              ml_feat["B16005_035E"] + ml_feat["B16005_039E"] + ml_feat["B16005_040E"] + \
                                              ml_feat["B16005_044E"] + ml_feat["B16005_045E"] 

In [6]:
#Separate geographical units in acs data for joining / clarity 
ml_feats = ml_feat.reset_index()
ml_feats["County"] = ml_feats['index'].apply(lambda x: str(x).replace(":", ",").split(",")[0])
ml_feats["State"] = ml_feats['index'].apply(lambda x: str(x).replace(":", ",").split(",")[1].replace(" ", ""))

In [7]:
#Retain only necessary features for model 
ml_feats = ml_feats[["County", "State", "GEO_ID", "Total Population", "Minority Population", 'ratio_income_poverty (above 200% FPL)', 
                    'Total Bachelors Degree', "Est Speak English Less than Well", "Per Capita Income"]]

### 2. Merge Census Data with other socioeconomic datasets

In [8]:
#read in and modify health insurance data; join to acs 
health_insur = pd.read_csv("health_insur.csv", dtype={'S2701_C02_001E':'Int64',
                                                     'S2701_C03_001E':np.float64, 
                                                      "GEO_ID": "str"})

health_insur = health_insur[['GEO_ID', 'S2701_C02_001E', 'S2701_C03_001E']]

health_cols = {'S2701_C02_001E': "Est Civilians Insured", 
              'S2701_C03_001E': "Est Percent Civilians Insured"}

health_insur.rename(columns=health_cols, inplace=True)

feature_mat = ml_feats.join(health_insur.set_index("GEO_ID"), on='GEO_ID', how="left")

C:\Users\richa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (36,37,40,41,46,47,50,51,64,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,230,231,234,235,240,241,244,245,250,251,254,255,280,281,284,285,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,441,442,445,446,481,482,485,486,542,543,544,545,5

In [9]:
#Read in and modify vehicle data; join to feature matrix 
vehicle_avail = pd.read_csv("vehicle_avail.csv", dtype={"B25044_001E": "Int64", 
                                                        "B25044_003E": "Int64",
                                                        "B25044_010E": "Int64"})
vehicle_avail['Percent Households without Vehicle'] = (vehicle_avail["B25044_003E"] + vehicle_avail["B25044_010E"]) / vehicle_avail["B25044_001E"]

vehicle = vehicle_avail[['GEO_ID', 'Percent Households without Vehicle']]

feature_mat = feature_mat.join(vehicle.set_index("GEO_ID"), on='GEO_ID', how="left")

In [10]:
#Read in and modify room occupancy data; join to feature matrix 
occupants = pd.read_csv("occupants_per_room.csv", dtype={"B25014_005E": "Int64", 
                                                        "B25014_006E": "Int64",
                                                        "B25014_007E": "Int64", 
                                                        "B25014_001E": "Int64"})
occupants['Household Level - More People than Rooms (%)'] = (occupants['B25014_005E'] + occupants['B25014_006E'] + occupants['B25014_007E']) / occupants['B25014_001E']


occupants = occupants[["GEO_ID", 'Household Level - More People than Rooms (%)']]
feature_mat = feature_mat.join(occupants.set_index("GEO_ID"), on='GEO_ID', how="left")

In [11]:
#Write feature matrix to csv 
feature_mat.to_csv("acs_features.csv", index=False)

In [12]:
#prep acs data file for merging with county health rankings file and CDC deaths data file

In [13]:
acs_df = pd.read_csv('acs_features.csv')
acs_df["acs_county_code"]=acs_df["GEO_ID"].str[-5:]

In [20]:
ch_rankings_df = pd.read_csv('2020_County_Health_Rankings.csv',dtype=str)
string_fips = ch_rankings_df['FIPS']

ch_rankings_df2 = pd.read_csv('2020_County_Health_Rankings.csv')
del ch_rankings_df2['FIPS']
ch_rankings = pd.concat([string_fips,ch_rankings_df2],axis=1)

In [21]:
merged_df = pd.merge(acs_df,ch_rankings,how='inner',left_on='acs_county_code', right_on='FIPS')

### 3. Prep and Merge CDC Covid death data 

In [22]:
#prep CDC COVID-19 deaths data for merging
cdc_deaths = pd.read_csv('CDC_county_deaths.csv')

def add_zero(i):
    j = " "
    if len(str(i)) == 4:
        j = "0"+str(i)
    else:
        j = str(i)
    return j


In [23]:
cdc_deaths['new_FIPS']=cdc_deaths['FIPS County Code'].map(add_zero)

In [24]:
final_merged_df = pd.merge(merged_df,cdc_deaths,how='inner',left_on='acs_county_code', right_on='new_FIPS')

In [26]:
#adding state code column for cross-validation
final_merged_df['state_code']=final_merged_df["new_FIPS"].str[:2]

In [28]:
final_merged_df.to_csv('merged_county_data.csv')